## Build Model

In [11]:
cat > ./model.stan << EOF
data {
    int <lower=1> NUM_DATA;
    int <lower=1> NUM_FRAG;
    int <lower=1> NUM_SEG;
    int <lower=1> NUM_SEG_PER_FRAG;
    int <lower=1> NUM_REP;
    
    int <lower=0> OUTPUT[NUM_DATA];
    int <lower=1> INPUT[NUM_DATA];
    real <lower=0> PHI;
    
    int L_DNA; // DNA library size
    int L_RNA; // RNA library size
    real shrinkageVar; // variance of shrinkage prior on effect sizes
}

transformed data {
   real l_rna=L_RNA; real l_dna=L_DNA;
   real LIB_RATIO=l_rna/l_dna;
}

parameters {
    real<lower=0.0001> gamma[NUM_SEG]; // effect size of each fragment
}

model {
    // Compute prefix-sum array for log(gamma)
    real PSA[NUM_SEG + 1];
    PSA[1]=0;
    for(i in 2:(NUM_SEG+1)){
        PSA[i]=PSA[i-1]+log(gamma[i-1]);
    }
    
    // shrinkage prior on effect sizes
    for(i in 1:NUM_SEG){
        gamma[i] ~ lognormal(0, shrinkageVar);    
    }
    
    // Likelihood
    for (i in 1:NUM_FRAG) {
        
        // gather effect from corresponding segments
        real gammaProd = exp(PSA[i + NUM_SEG_PER_FRAG] - PSA[i]);
        
        // mean(output) := segment effect x input x library ratio
        for (k in 1:NUM_REP) {
            int idx_data = NUM_FRAG * (i - 1) + k;
            OUTPUT[idx_data] ~ neg_binomial_2(INPUT[idx_data] * LIB_RATIO * gammaProd, PHI);
            
            //print(seg_effect, " ", INPUT[idx_data], " ", INPUT[idx_data] * seg_effect);
        } 
    }
}
EOF

**Check**

In [12]:
cat ./model.stan

data {
    int <lower=1> NUM_DATA;
    int <lower=1> NUM_FRAG;
    int <lower=1> NUM_SEG;
    int <lower=1> NUM_SEG_PER_FRAG;
    int <lower=1> NUM_REP;
    
    int <lower=0> OUTPUT[NUM_DATA];
    int <lower=1> INPUT[NUM_DATA];
    real <lower=0> PHI;
    
    int L_DNA; // DNA library size
    int L_RNA; // RNA library size
    real shrinkageVar; // variance of shrinkage prior on effect sizes
}

transformed data {
   real l_rna=L_RNA; real l_dna=L_DNA;
   real LIB_RATIO=l_rna/l_dna;
}

parameters {
    real<lower=0.0001> gamma[NUM_SEG]; // effect size of each fragment
}

model {
    // Compute prefix-sum array for log(gamma)
    real PSA[NUM_SEG + 1];
    PSA[1]=0;
    for(i in 2:(NUM_SEG+1)){
        PSA[i]=PSA[i-1]+log(gamma[i-1]);
    }
    
    // shrinkage prior on effect sizes
    for(i in 1:NUM_SEG){
        gamma[i] ~ lognormal(0, shrinkageVar);    
    }
    
    // Likelihood
    for (i in 1:NUM_FRAG) {
        
        // gather effect from corresponding segments
        r

**Compile model**

In [13]:
source ../config.sh
FD_CUR=$(pwd)

make -C ${STAN_PATH} ${FD_CUR}/model

make: Entering directory '/work/kk319/exe/cmdstan'

--- Translating Stan model to C++ code ---
bin/stanc  --o=/hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks/12_model_sliding/model.hpp /hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks/12_model_sliding/model.stan

--- Compiling, linking C++ code ---
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include   -O3 -I src -I stan/src -I lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.9 -I stan/lib/stan_math/lib/boost_1.75.0 -I stan/lib/stan_math/lib/sundials_5.7.0/include    -DBOOST_DISABLE_ASSERTS         -c -Wno-ignored-attributes   -x c++ -o /hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks/12_model_sliding/model.o /hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks/12_model_sliding/model.hpp
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I st

In [3]:
## set environment
FD_WRK=./
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_trivial_v1.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./
FD_INP=${FD_WRK}/input
FD_OUT=${FD_WRK}/output
${FD_WRK}/model sample \
    thin=1 num_samples=1000 num_warmup=100 \
    init=${FD_INP}/init.json \
    data   file=${FD_INP}/input_trivial_v1.json \
    output file=${FD_OUT}/output_trivial_v1.csv
EOF

Submitted batch job 15669650


In [4]:
cat ${FD_LOG}/model_trivial_v1.txt

method = sample (Default)
  sample
    num_samples = 1000 (Default)
    num_warmup = 100
    save_warmup = 0 (Default)
    thin = 1 (Default)
    adapt
      engaged = 1 (Default)
      gamma = 0.050000000000000003 (Default)
      delta = 0.80000000000000004 (Default)
      kappa = 0.75 (Default)
      t0 = 10 (Default)
      init_buffer = 75 (Default)
      term_buffer = 50 (Default)
      window = 25 (Default)
    algorithm = hmc (Default)
      hmc
        engine = nuts (Default)
          nuts
            max_depth = 10 (Default)
        metric = diag_e (Default)
        metric_file =  (Default)
        stepsize = 1 (Default)
        stepsize_jitter = 0 (Default)
id = 0 (Default)
data
  file = .//input/input_trivial_v1.json
init = .//input/init.json
random
  seed = 4021865561 (Default)
output
  file = .//output/output_trivial_v1.csv
  diagnostic_file =  (Default)
  refresh = 100 (Default)
  sig_figs = -1 (Default)
  profile_file = profile.csv (Default)


Gradient evaluation took 4.

In [8]:
## set environment
FD_WRK=./
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_trivial_v2.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./
FD_INP=${FD_WRK}/input
FD_OUT=${FD_WRK}/output
FD_MOD=${FD_WRK}/model

${FD_MOD}/model sample \
    thin=1 num_samples=1000 num_warmup=100 \
    init=${FD_INP}/init.json \
    data   file=${FD_INP}/input_trivial_v2.json \
    output file=${FD_OUT}/output_trivial_v2.csv
EOF

Submitted batch job 15673701


In [9]:
cat ${FD_LOG}/model_trivial_v2.txt

method = sample (Default)
  sample
    num_samples = 1000 (Default)
    num_warmup = 100
    save_warmup = 0 (Default)
    thin = 1 (Default)
    adapt
      engaged = 1 (Default)
      gamma = 0.050000000000000003 (Default)
      delta = 0.80000000000000004 (Default)
      kappa = 0.75 (Default)
      t0 = 10 (Default)
      init_buffer = 75 (Default)
      term_buffer = 50 (Default)
      window = 25 (Default)
    algorithm = hmc (Default)
      hmc
        engine = nuts (Default)
          nuts
            max_depth = 10 (Default)
        metric = diag_e (Default)
        metric_file =  (Default)
        stepsize = 1 (Default)
        stepsize_jitter = 0 (Default)
id = 0 (Default)
data
  file = .//input/input_trivial_v2.json
init = .//input/init.json
random
  seed = 4028370519 (Default)
output
  file = .//output/output_trivial_v2.csv
  diagnostic_file =  (Default)
  refresh = 100 (Default)
  sig_figs = -1 (Default)
  profile_file = profile.csv (Default)


Gradient evaluation took 4.

In [10]:
## set environment
FD_WRK=./
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_trivial_v3.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./
FD_INP=${FD_WRK}/input
FD_OUT=${FD_WRK}/output
FD_MOD=${FD_WRK}/model

${FD_MOD}/model sample \
    thin=1 num_samples=1000 num_warmup=100 \
    init=${FD_INP}/init.json \
    data   file=${FD_INP}/input_trivial_v3.json \
    output file=${FD_OUT}/output_trivial_v3.csv
EOF

Submitted batch job 15673710


In [11]:
cat ${FD_LOG}/model_trivial_v3.txt

method = sample (Default)
  sample
    num_samples = 1000 (Default)
    num_warmup = 100
    save_warmup = 0 (Default)
    thin = 1 (Default)
    adapt
      engaged = 1 (Default)
      gamma = 0.050000000000000003 (Default)
      delta = 0.80000000000000004 (Default)
      kappa = 0.75 (Default)
      t0 = 10 (Default)
      init_buffer = 75 (Default)
      term_buffer = 50 (Default)
      window = 25 (Default)
    algorithm = hmc (Default)
      hmc
        engine = nuts (Default)
          nuts
            max_depth = 10 (Default)
        metric = diag_e (Default)
        metric_file =  (Default)
        stepsize = 1 (Default)
        stepsize_jitter = 0 (Default)
id = 0 (Default)
data
  file = .//input/input_trivial_v3.json
init = .//input/init.json
random
  seed = 4028571590 (Default)
output
  file = .//output/output_trivial_v3.csv
  diagnostic_file =  (Default)
  refresh = 100 (Default)
  sig_figs = -1 (Default)
  profile_file = profile.csv (Default)


Gradient evaluation took 4.

In [13]:
## set environment
FD_WRK=./
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_trivial_v4.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./
FD_INP=${FD_WRK}/input
FD_OUT=${FD_WRK}/output
FD_MOD=${FD_WRK}/model

${FD_MOD}/model sample \
    thin=1 num_samples=1000 num_warmup=100 \
    init=${FD_INP}/init.json \
    data   file=${FD_INP}/input_trivial_v4.json \
    output file=${FD_OUT}/output_trivial_v4.csv
EOF

Submitted batch job 15673733
